In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

# metadata
print(adult.metadata)

# variable information
print(adult.variables)



In [3]:
# importing libraries. Run this first!!

import pandas as pd
import numpy as np

## Dataset Analysis

### About
We chose the [Adult 2 dataset](https://archive.ics.uci.edu/dataset/2/adult) from UCI's Machine Learning Repository. This dataset compiles US Census information relevant to predicting income. It will be used to predict if a sample has an income over 50k USD a year.

### Does it comply with the requirements?
Adult 2 contains 48,000 rows, so it has over 2,000 rows. It contains 14 features, however some will not be used. We will still consider at least 10. Finally, there are several opportunities for encountering data leakage. 

First, a feature *fnl_wgt* computes how many rows in the dataset are like this. Frequency of a specific sample considers every sample in the set, which means this feature is not individual to the sample. Therefore, we will not consider this column during training.

Second, there are several duplicate entries in the dataset. We must make each sample singular to give each sample equal weighting. 

Finally, there are two columns pertaining to education: a string representation and an enumeration of the string options. Both should not be used, as it may give that feature unequal weighting, and samples with unknown education backgrounds would be minimized.

Therefore, this dataset is compliant with the project outlines.

In [8]:
# this block will drop some columns

dropped_columns = [
    'fnlwgt',
    'education-num'
]

new_data = X.drop(columns=dropped_columns)

print(new_data.columns)

# now new_data has all the right variables

Index(['age', 'workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
       'hours-per-week', 'native-country'],
      dtype='str')


Removing duplicates

In [ ]:
# drops duplicate entries

print(len(new_data))
new_data = new_data.drop_duplicates()
print(len(new_data))

48842
41533
